In [1]:
##############################
## CREATE COUNTABLE BINARY ARRAY with field view !!!!!!
#############################

import numpy as np
import scipy.misc as smp
import random



arr_size_half = 20
max_N = 1
view_size_half = 3
object_value = 100.

def create_inp_outp_array():
    max_objects = 4
    n_ones = random.randint(1, max_objects)
    one_positions = random.sample(range(view_size_half, view_size_half + max_objects), n_ones)
    binary_array = np.zeros(max_objects+2*view_size_half)
    binary_array[one_positions] = object_value
    #binary_array[2] = object_value
    binary_array[0] = 9
    binary_array[-1] = 9

    return binary_array, n_ones
    

 

def make_data_set(n_samples):
  mult_inp, mult_out = create_inp_outp_array()
  for i in range(n_samples - 1):
      single_inp, single_out = create_inp_outp_array()
      mult_inp = np.vstack([mult_inp, single_inp])
      mult_out = np.vstack((mult_out, single_out))
  return mult_inp, mult_out



train_input, train_output = make_data_set(10000)
max_N=20
test_input, test_output = make_data_set(10000)


#Print single one
inp_ex, out_ex = make_data_set(10)
print("inp_ex", inp_ex)
print("out_ex", out_ex)


#Print set data
print("===========================")
print("train_input: number of examples: ", train_input.shape)
print("train_input: number of examples: ", train_input[:,0].size)
print("train_input: single example size: ", train_input[0].size)
print("train_input: dim of elements in example: ", train_input[0,0].size)




inp_ex [[  9.   0.   0.   0. 100. 100. 100.   0.   0.   9.]
 [  9.   0.   0.   0.   0. 100.   0.   0.   0.   9.]
 [  9.   0.   0.   0.   0. 100.   0.   0.   0.   9.]
 [  9.   0.   0. 100. 100.   0. 100.   0.   0.   9.]
 [  9.   0.   0. 100. 100. 100. 100.   0.   0.   9.]
 [  9.   0.   0. 100. 100.   0. 100.   0.   0.   9.]
 [  9.   0.   0.   0. 100. 100. 100.   0.   0.   9.]
 [  9.   0.   0. 100.   0. 100.   0.   0.   0.   9.]
 [  9.   0.   0.   0.   0.   0. 100.   0.   0.   9.]
 [  9.   0.   0. 100. 100. 100. 100.   0.   0.   9.]]
out_ex [[3]
 [1]
 [1]
 [3]
 [4]
 [3]
 [3]
 [2]
 [1]
 [4]]
train_input: number of examples:  (10000, 10)
train_input: number of examples:  10000
train_input: single example size:  10
train_input: dim of elements in example:  1


In [0]:
########################################
## UPDATE ENVIRONMENT WHEN MOVING, PICKING WITH LIMITED FIELD VIEW
##################################################################


def update_env(state, pos, action, reward,t):
  
  push_finish_button = False
  reward=0.0
  if(action==0):                      #[0] ----> pick the one at the current position
    if(state[pos]==object_value):
      state[pos]= 0
      reward += point_reward
    if(state[pos]==0):
      reward += wrong_point_reward     
  elif(action==1):                    #[1] ----> stop the current counting process
    push_finish_button = True
  elif(action==2):                    #[2] ----> go to the left
    if(pos>view_size_half):
      pos-=1
  elif(action==3):                    #[3] ----> go to the right
     if(pos<state.size - view_size_half-1):
      pos+=1
  else:
    ("How could your action go beyond N+3?")
    
  if(push_finish_button==False):
    reward += time_penalty_reward
  if (push_finish_button==True or t==trial_duration-1):
    if(state[1]==0 and state[2]==0):
      reward += final_reward
  if(state[1]==0 and state[2]==0):
      reward += final_reward
      push_finish_button = True
    
  return state, pos, push_finish_button, reward 
    
    
def get_field_view(state, pos, view_size_half):
  fiew_field =  copy.copy(state[pos-view_size_half:pos+view_size_half+1])
  return fiew_field

def action_int_to_letter(action):
  
  if(action==0):          #[0] ----> pick the one at the current position
     act_str = "P"
  elif(action==1):        #[1] ----> stop the current counting proces
     act_str = "STOP"
  elif(action==2):        #[2] ----> go to the left
    act_str = "L"
  elif(action==3):        #[3] ----> go to the right
    act_str = "R"
  else:
    ("How could your action go beyond N+3?")
    
  return act_str
    

def get_animation_pic(state,pos,view_size_half,action):
  s = " "
  for i in range(pos-view_size_half):
    s+="   "
  s += "|"
  for i in range(view_size_half):
    s+="__"
  s += "I"
  for i in range(view_size_half):
    s+="__"
  s += "|"
  for i in range(state.size - (pos-view_size_half+3) ):
    s+="   "
  s+= "  "
  act_str = action_int_to_letter(action)
  s+= act_str
  print(state)
  print(s)
  
  

  
class ExperienceBuffer:
  
  def __init__(self):
    self.observation = np.array([]).reshape(0,dim_obs)
    self.new_observation = np.array([]).reshape(0,dim_obs)
    self.action = np.empty(0)
    self.push_finish_button = np.empty(0)
    self.reward = np.empty(0)
  
  
  def add_exp(self, observation, new_observation, action, reward, push_finish_button):
  
    if(len(exp_buffer.observation) < batch_size):  

        self.observation = np.vstack([exp_buffer.observation, observation])
        self.new_observation = np.vstack([exp_buffer.new_observation, new_observation])
        self.action = np.append(exp_buffer.action, action)
        self.reward = np.append(exp_buffer.reward, reward)
        self.push_finish_button = np.append(exp_buffer.push_finish_button, np.float32(push_finish_button))

    else:
        rand_i = random.randint(0,batch_size-1)
        self.observation[rand_i] = observation
        self.new_observation[rand_i] = new_observation
        self.action[rand_i] = action
        self.reward[rand_i] = reward
        self.push_finish_button[rand_i] = np.float32(push_finish_button)
        
    
  
  

In [0]:
########################
## SET PARAMETERS
########################
# RL Parameters

point_reward = +0.1
wrong_point_reward = -0.0
time_penalty_reward = -0.05
final_reward = +1.0
stop_reward = +0.0
trial_duration = 10


# NETWORK LEARNING PARAMETERS
batch_size = 1
buffer_size = 1
gamma = 1.0
epsilon_init = 1.
learning_rate_init = 3e-4
epsi_decay = .999
lr_decay = .999
epsilon_min = 0.01
learning_min = 0.00001

In [30]:
#!/usr/bin/env python

#import gym
import numpy as np
import numpy.random as rd
import matplotlib.pyplot as plt
#from gym.envs.classic_control.cartpole import CartPoleEnv
#from cartpole_utils import plot_results,print_results
import tensorflow as tf
import copy
import matplotlib
import matplotlib.pyplot as plt


def init_weights(shape, name):
    return tf.Variable(tf.random_normal(shape, stddev=0.01)/ np.sqrt(dim_obs), name=name)


def Q_network(state_holder, w_h, w_h2, w_o, b, b_h):
    h = tf.nn.relu(tf.matmul(state_holder, w_h) + b_h )
    #h2 = tf.nn.relu(tf.matmul(h, w_h2))
    Q =  tf.matmul(h, w_o, name='output_activation') + b 
    #Q = tf.nn.sigmoid(a_z, name='Q_model')
    return Q
  
def Q_RNN_Network(cell, obs_holder, net_state_sess, weight, bias):
  val, next_net_state = tf.nn.dynamic_rnn(cell, obs_holder, initial_state=net_state_sess, dtype=tf.float32)
  val = tf.transpose(val, [1, 0, 2])
  last = tf.gather(val, int(val.get_shape()[0]) - 1)
  Q_i = tf.matmul(last, weight) + bias
  return Q_i, next_net_state





# Load Input=Start-of-a-game &&  Classes=Check-for-reward
trX = train_input
trY = train_output
teX = test_input
teY = test_output



## SET ENVIRONMENT + #ofActions #ofStates
n_action = 4     #[Pick, Stop, Left, Right]
dim_obs = 1+2*view_size_half







##############################
## BUILD TENSORFLOW
#############################


# PLACEHOLDER
action_holder = tf.placeholder(dtype=tf.int32, shape=(None, 1), name="action_holder")  # +1 because that is the node responsible to say if it is done ore not
state_holder = tf.placeholder(dtype=tf.float32, shape=(None, dim_obs), name='state_holder')
next_state_holder = tf.placeholder(dtype=tf.float32, shape=(None, dim_obs), name='next_state_holder')


#PLACEHOLDER
r_holder = tf.placeholder(dtype=tf.float32, shape=(None, 1), name='r_holder')
is_done_holder = tf.placeholder(dtype=tf.float32, shape=(None, 1), name='is_done_holder')
learning_rate_holder = tf.placeholder(dtype=tf.float32, name='symbolic_state')


# Q - NETWORK
n_h_neurons = 20
w_h = init_weights([dim_obs, n_h_neurons], "w_h")
w_h2 = init_weights([n_h_neurons, n_h_neurons], "w_h2")
w_o = init_weights([n_h_neurons, n_action], "w_o")
b0 = np.zeros(n_action, dtype=np.float32)
b_h0 = np.zeros(n_h_neurons, dtype=np.float32)
b = tf.Variable(initial_value=b0, trainable=True, name='bias')
b_h = tf.Variable(initial_value=b_h0, trainable=True, name='bias')


# Q - RNN NETWORK
# Initialize the parameters of the Q-RNN model
num_hidden = 20
weight = tf.Variable(tf.truncated_normal([num_hidden, n_action]))
bias = tf.Variable(tf.constant(0.1, shape=[n_action]))

cell = tf.contrib.rnn.LSTMCell(num_hidden,state_is_tuple=True,reuse=tf.AUTO_REUSE)

net_state_sess = cell.zero_state(batch_size, tf.float32)
next_net_state = cell.zero_state(batch_size, tf.float32)


#############################


###################
### CALCULATE Q
################
Q = Q_network(state_holder, w_h, w_h2, w_o,b, b_h)
next_Q = Q_network(next_state_holder, w_h, w_h2, w_o,b, b_h)
#Q, next_net_state = Q_RNN_Network(cell, obs_holder, net_state_sess, weight, bias)
#next_Q = Q_RNN_Network(cell, next_obs_holder, net_state_sess, weight, bias)



### COMPARE REWARD<->PREDICTED REWARD ===> LOSS ####

#### From https://lilianweng.github.io/lil-log/2018/05/05/implementing-deep-reinforcement-learning-models.html
# The prediction by the primary Q network for the actual actions.
action_one_hot = tf.one_hot(action_holder, n_action, 1.0, 0.0, name='action_one_hot')
R = tf.reduce_sum(Q * action_one_hot, reduction_indices=-1, name='q_acted')

# The optimization target defined by the Bellman equation and the target network.
max_q_next_by_target = tf.reduce_max(next_Q, axis=-1)
R_next = r_holder + (1. - is_done_holder) * gamma * max_q_next_by_target

# The loss measures the mean squared error between prediction and target.
err = tf.reduce_mean(tf.square(R - tf.stop_gradient(R_next)), name="loss_mse_train")
training_step = tf.train.AdamOptimizer(learning_rate=learning_rate_holder).minimize(err, name="adam_optim")

###########################################################################################





sess = tf.Session()  # FOR NOW everything is symbolic, this object has to be called to compute each value of Q
sess.run(tf.initialize_all_variables())

################################
################################





################################
## PI(S)
## First check if its random time THEN
        # - calculate State--Network--> Q
        # - take maximum of Q
        # - just remember and return index of maximum    ---> returns ACTION that leads to max_Q-value (not value itself)
################################

def policy(observation):
    Q_values = sess.run(Q, feed_dict={state_holder: observation.reshape(1, dim_obs)} )
    #Q_values, net_state = sess.run((Q, next_net_state), feed_dict={obs_holder: obs.reshape(1, dim_obs,1), net_state_sess: net_state})

    Q_values_reduced = Q_values[0, :]
    val = np.max(Q_values_reduced)  # Change in dimension
    max_indices = np.where(Q_values_reduced == val)[0]
    
    action = 0
    if rd.rand() < epsilon:
      action = rd.randint(0, n_action)
    else:
      action = rd.choice(max_indices)

    return action 










##############################
## ACTUAL RUN
###########################

time_list = []
reward_list = []
err_list = []
val_list = []
trial_err_list = []
mean_reward_list = []

mean_reward = 0
print_every_n = 50
epsilon = epsilon_init
learning_rate = learning_rate_init
n_episodes = trX[:, 0].size




  
exp_buffer = ExperienceBuffer()


for k in range(n_episodes):

    acc_reward = 0                  # Init the accumulated reward
    push_finish_button = False
    internal_counter_memory = 0
    
    # Initialize external and internal state of agent
    k_n = rd.randint(0, trX[:,0].size)                                            ######   !!!!!!!!!!!!!!!!!
    state = copy.copy(trX[k_n, :])
    pos = view_size_half
    observation = get_field_view(state, pos, view_size_half)
    net_state = sess.run(net_state_sess)
        
    
    for t in range(trial_duration):  
        

        old_observation = copy.copy(observation)
        old_internal_counter_memory = copy.copy(internal_counter_memory)
        state_copy = copy.copy(state)

        #####################
        ## Chose action by Random or Maximum Q-value
        ################################
          
        action = policy(observation)
          
        ######################
        ## Update Environment, get reward according to chosen action
        #################################################    
        new_state, pos, push_finish_button, reward = update_env(state_copy, pos, action, reward, t)
        new_observation = get_field_view(new_state, pos, view_size_half)
        exp_buffer.add_exp(observation, new_observation, action, reward, push_finish_button)


        
        ############################
        ## TRAINING STEP, ERROR CALCULATION (RUN SESSION)
        ############################
                
        
        buffer_size = exp_buffer.observation[:,0].size
        
        if(buffer_size >= batch_size):
     
          error, R_out, R_next_out = sess.run([err, R, R_next], feed_dict={
              state_holder: exp_buffer.observation.reshape(batch_size, dim_obs),
              next_state_holder: exp_buffer.new_observation.reshape(batch_size, dim_obs) ,
              action_holder: exp_buffer.action.reshape(batch_size, 1),
              is_done_holder: exp_buffer.push_finish_button.reshape(batch_size, 1),
              r_holder: exp_buffer.reward.reshape(batch_size, 1)})   

          sess.run(training_step, feed_dict={
              state_holder: exp_buffer.observation.reshape(batch_size, dim_obs),
              next_state_holder: exp_buffer.new_observation.reshape(batch_size, dim_obs),
              action_holder: exp_buffer.action.reshape(batch_size, 1),
              is_done_holder: exp_buffer.push_finish_button.reshape(batch_size, 1),
              r_holder: exp_buffer.reward.reshape(batch_size, 1),
              learning_rate_holder: learning_rate})
        

          if(epsilon>epsilon_min):
             epsilon *= epsi_decay
          if (learning_rate > learning_min):
             learning_rate *= lr_decay
        

        observation = new_observation  # Pass the new state to the next step
        old_state = copy.copy(state)
        state = new_state
        acc_reward += reward  # Accumulate the reward
        #trial_err_list.append(error)

        
        ########################
        ## PRINT IN SINGLE TIME STEPS
        #########################
                    
        if( (k>=0 and k<9) and (k>=700 and k<709) ):
          if(t==0):
            print("\n ================")
            print("K= ", k)
          print("t: ", t)
          print("Action", action)
          get_animation_pic(state,pos,view_size_half,action)
          print("observation 2: ", observation)      
          print("Reward: ", reward)
          print("--------------")
         
        if (push_finish_button==True or t==trial_duration-1):
          mean_reward += acc_reward
          break
        

    ########################
    ## PRINT MEAN AFTER EVERY N TIME STEPS
    ###################################
    if (k % print_every_n == 0):
        mean_reward = float(mean_reward) / print_every_n
        print("K", k)
        print(k, "th mean_reward=", mean_reward)
        print("Epsilon: ", epsilon)
        mean_reward_list.append(mean_reward)
        mean_reward = 0.0
         

    #err_list.append(np.mean(trial_err_list))
    time_list.append(t + 1)
    reward_list.append(acc_reward)  # Store the result


    
    
    
    
    
    
print("mean_reward_list", mean_reward_list)
reward_file = 'Mean_rewards/'  + str(n_h_neurons) + '_neurons_' + str(n_action) + '_max_objects'
np.save(reward_file, mean_reward_list)

matplotlib.get_backend()
legend_string = str(n_h_neurons) + " neurons/hidden layer"
plt.plot(np.arange(0,len(mean_reward_list)), mean_reward_list, label = legend_string)
plt.legend()
print("legend_string= ", legend_string)
plt.ylabel('Success rate')
plt.xlabel('Episodes')
plt.ylim(0,1)
title_string = 'Success rate for agent when counting binaries between 0 and ' + str(n_action) + ' binaries'
plt.title(title_string)
plt.savefig('Q_learning_binaries.png')
plt.show()


K 0
0 th mean_reward= 0.04
Epsilon:  0.999
K 50
50 th mean_reward= 1.1860000000000004
Epsilon:  0.9502544225688344
K 100
100 th mean_reward= 1.1190000000000007
Epsilon:  0.9038873549665959
K 150
150 th mean_reward= 1.2790000000000004
Epsilon:  0.8597827393003539
K 200
200 th mean_reward= 1.4010000000000007
Epsilon:  0.8178301806491574
K 250
250 th mean_reward= 1.3820000000000008
Epsilon:  0.7779246707428734
K 300
300 th mean_reward= 1.4410000000000007
Epsilon:  0.7399663251239436
K 350
350 th mean_reward= 1.4720000000000004
Epsilon:  0.7038601331341691
K 400
400 th mean_reward= 1.4660000000000004
Epsilon:  0.6695157201007336
K 450
450 th mean_reward= 1.4410000000000005
Epsilon:  0.6368471211262058
K 500
500 th mean_reward= 1.5230000000000006
Epsilon:  0.6057725659163237
K 550
550 th mean_reward= 1.5480000000000007
Epsilon:  0.576214274106964
K 600
600 th mean_reward= 1.659
Epsilon:  0.548098260578011
K 650
650 th mean_reward= 1.5270000000000001
Epsilon:  0.5213541502668072
K 700
700 th

KeyboardInterrupt: ignored

In [0]:
a=5
b=a+2
print("b",b)
a=7
print("b",b)

('b', 7)
('b', 7)


In [0]:
class ExperienceBuffer:
  observation = []
  
exp_buffer = ExperienceBuffer()

exp_buffer.observation.append(5)
exp_buffer.observation.append(7)

print(exp_buffer.observation)
print("Length", len(exp_buffer.observation))

[5, 7]
('Length', 2)


In [0]:
class ExperienceBuffer:
  observation = np.empty(0)
  
exp_buffer = ExperienceBuffer()

exp_buffer.observation = np.append(exp_buffer.observation,5)
exp_buffer.observation = np.append(exp_buffer.observation,7)
exp_buffer.observation = np.append(exp_buffer.observation,9)
exp_buffer.observation = np.append(exp_buffer.observation,1)

print(exp_buffer.observation)
print("Length", exp_buffer.observation.size)


randy_arr = np.random.choice(exp_buffer.observation,2)
print("randy_arr= ", randy_arr)

[5. 7. 9. 1.]
('Length', 4)
('randy_arr= ', array([1., 7.]))


In [0]:
class ExperienceBuffer:
  observation = np.array([]).reshape(0,3)
  
exp_buffer = ExperienceBuffer()

exp_buffer.observation = np.vstack([exp_buffer.observation, [1,2,3]])
exp_buffer.observation = np.vstack([exp_buffer.observation, [9,9,9]])
exp_buffer.observation = np.vstack([exp_buffer.observation, [4,4,4]])
exp_buffer.observation = np.vstack([exp_buffer.observation, [1,1,1]])
buffer_size = exp_buffer.observation[:,0].size

print(exp_buffer.observation)
print("Length", buffer_size)

batch_size = 2

#randy_arr = np.random.choice(exp_buffer.observation,2)
#print("randy_arr= ", randy_arr)

#indy = [0,2]
indy = random.sample(range(buffer_size), batch_size)
print("selected vectors: ", exp_buffer.observation[indy])

[[1. 2. 3.]
 [9. 9. 9.]
 [4. 4. 4.]
 [1. 1. 1.]]
('Length', 4)
('selected vectors: ', array([[4., 4., 4.],
       [1., 2., 3.]]))


In [0]:
a = np.empty(0)
a = np.append(a, 1)
b=a.reshape(batch_size, 1)
print("b: ", b)

b:  [[1.]]


In [0]:
#a = random.sample(range(1), 1)
a= range(1)
b= random.sample(a, 1)
print("a= ", a )
print("b= ", b )

a=  range(0, 1)
b=  [0]


In [0]:
class Restaurant(object):
    bankrupt = False
    def open_branch(self):
        if not self.bankrupt:   #self.bankrupt
            print("branch opened")
            
x = Restaurant()
y = Restaurant()
y.bankrupt = True

print("x.bankrupt", x.bankrupt)
x.open_branch()

x.a = 5


x.bankrupt False
branch opened
